# REST request to Retrieve real-time Hydrogeological Data, an example

________
This Jupyter Notebook serves as an example of how to retrieve hydrogeological data in real-time using REST requests. It is a useful tool for accessing open data from the aquifer of interest, such as piezometric levels, temperature, and electrical conductivity.

For this purpose, we illustrate how to retrieve hydrogeological data for the aquifers of Catalonia, sourced from the data generated by the [Catalan Water Agency](https://aca.gencat.cat/ca/inici/index.html#googtrans(ca|en)) and stored in the open sensor platform [Sentilo](https://www.sentilo.io/).

The data can be accessed at:
- https://aca.gencat.cat/ca/laigua/consulta-de-dades/dades-obertes/dades-obertes-temps-real/index.html#googtrans(ca|en)

A user manual (in Catalan) on how to retrieve the data is available at:
- https://aca.gencat.cat/web/.content/20_Aigua/08_consulta_de_dades/01_dades_obertes/02_dades_obertes_temps_real/us_serveis_dades_API_REST.pdf

_____________

<br>

## Importing the needed libraries

In [1]:
#-- Check and install required packages if not already installed
import sys
import subprocess

def if_require(package):
    if (package) == "ipywidgets":
        try:
            __import__(package)
            import ipywidgets
            if ipywidgets.__version__ != "8.1.5":
                print("'ipywidget' version not compatible") 
                subprocess.check_call([sys.executable, "-m", "pip", "install", 
                                       "--force-reinstall", "-v", "ipywidgets"])      
        except ImportError:
            print(f"{package} not found. Installing...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])     
    else:
        try:
            __import__(package)
        except ImportError:
            print(f"{package} not found. Installing...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        
        
#-- Install packages only if they aren't already installed 
if_require("geopy")
if_require("geopandas")
if_require("mapclassify")
if_require("ipywidgets")
if_require("folium")
if_require("requests")

#-- Import the rest of libraries --#
import matplotlib.pyplot as plt
import ipywidgets as widgets
import geopandas as gpd
import pandas as pd
import numpy as np
import requests
import json
import re
import os

from geopy.geocoders import Nominatim 
from IPython.display import display
from shapely.geometry import Point 
from ipywidgets import *

import warnings
warnings.filterwarnings("ignore")

## Coding the functions and the REST API request

In [2]:
#-------------------------------------------------------------------#
# DEALING WITH THE JSON:                                            #
#                                                                   #
# (1) We clean it                                                   #
# (2) We generate 2 parse-response:                                 #
#       - for the whole catalog of points and one                   #
#       - for the sensors                                           #
#-------------------------------------------------------------------#

def clean_json_string(json_string):
    """Cleans a JSON string by removing or replacing problematic characters.
    
    This function removes invalid escape sequences from a JSON string and 
    replaces specific characters with alternatives.

    Parameters
    ----------
    json_string : str
        the json string from the request to clean

    Returns
    -------
    str
        the cleaned json
    """   
    #-- Remove invalid escape characters from the JSON string
    cleaned_string = re.sub(r'\\[^\\"bfnrtu]', '', json_string)   
    #-- Character \b5 is replaced by "m"
    cleaned_string = re.sub(r'\\b5', 'm', cleaned_string)
    #-- Character \ba is replaced by "º"
    cleaned_string = re.sub(r'\\ba', 'º', cleaned_string)
    
    return cleaned_string

def parse_response_catalog(url):
    """Fetches and parses JSON data from a given URL.
    
    This function sends a request to the provided URL, checks for a successful 
    response (status code 200), and decodes the content into JSON format. 
    If the content is not valid JSON, an error message is printed.

    Parameters
    ----------
    url : requests.Response
        A response object obtained from a `requests.get()` call

    Returns
    -------
    dict or None
        A dictionary parsed from the JSON content of the response if successful. 
        None if the response is unsuccessful or the JSON decoding fails.
    """  
    if url.status_code == 200:
        try:
            data = url.content.decode('utf-8')
            cleaned_data = clean_json_string(data)
            json_data = json.loads(cleaned_data)
            return json_data
        except json.JSONDecodeError as e:
            print("Error decoding JSON:", e)
    else:
        print("Unexpected response (%s: %s)." % (url.status_code, url.reason))
        return None
    

def parse_response_ob(sensor_name, url_sensor):
    """Fetches and parses sensor data from a given URL.

    This function sends a request to the specified sensor URL, checks for a successful
    response (status code 200), and decodes the content into JSON format. If the content 
    is not valid JSON, an error message is printed.

    Parameters
    ----------
    sensor_name : requests.Response
        The name of the sensor
    url_sensor : str
        A response object returned from a `requests.get()` call

    Returns
    -------
    dict or None
        A dictionary parsed from the JSON content of the response if successful. 
        None if the response is unsuccessful or the JSON decoding fails.
    """  
    if url_sensor.status_code == 200:
        try:
            data = url_sensor.content.decode('utf-8')
            cleaned_data = clean_json_string(data)
            json_ob = json.loads(cleaned_data)
            return json_ob
        except json.JSONDecodeError as e:
            print("Error decoding JSON:", e)
    else:
        print("Unexpected response (%s: %s)." % (url_sensor.status_code, url_sensor.reason))
        return None

#-------------------------------------------------------------------#
# UNDERSTANDING THE JSON:                                           #
#                                                                   #
# (1) We list the aquifers based on the JSON                        #
# (2) We retrieve the value(s) of the observation                   #
# (3) We print the value of the observation                         #
#-------------------------------------------------------------------#

def list_aquifers(json_data):
    """Lists available aquifers and their associated measurements from JSON data.

    This function processes a JSON dataset to extract unique aquifers and
    measurement descriptions.

    Parameters
    ----------
    json_data : dict
        A dictionary containing JSON data of aquifers and sensors.

    Returns
    -------
    tuple of lists
        A sorted list of unique aquifers and a sorted list of unique measurements.
    """    
    if json_data:
        providers = json_data["providers"]
        aq_set = set()
        descripcio_set = set()
        for provider in providers:
            sensors = provider["sensors"]
            for sensor in sensors:
                additional_info = sensor.get("componentAdditionalInfo", {})
                aquifer = additional_info.get("Aqüífer", "")
                aq_set.add(aquifer)
                mesura = sensor.get("description", "")
                descripcio_set.add(mesura)
        return sorted(list(aq_set)), sorted(list(descripcio_set))  

def value_ob(json_ob, sensor_name, url_sensor):
    """Updates the global OBS DataFrame with the value from a JSON response.

    This function checks the parsed JSON response (`json_ob`) for observations 
    and updates the 'Value' column in the global OBS DataFrame for the given `sensor_name`. 
    If no observation value is found, the sensor entry is removed from the OBS DataFrame.

    Parameters
    ----------
    json_ob : dict
        A dictionary parsed from the JSON content of the sensor's data
    sensor_name : str
        The name of the sensor for which the observation value should be updated.
    url_sensor : requests.Response
        The response object from the request used to fetch the sensor's data.
    """
    global OBS
    if json_ob and json_ob.get('observations', []):
        value_ob = json_ob['observations'][0]['value']
        OBS.loc[OBS['Sensor'] == sensor_name, 'Value'] = value_ob  
    else:
        OBS = OBS[OBS['Sensor'] != sensor_name]  

def to_print(json_data, aq, mes, save):
    """Displays and optionally saves measurement data for a selected aquifer.

    This function processes JSON data to filter sensors for a specified aquifer 
    and measurement type. It displays the data in a table and optionally saves 
    it to a CSV file. If no data is found for the measurement, it notifies the user.

    Parameters
    ----------
    json_data : dict
        The JSON data containing information about sensors
    aq : str
        The name of the selected aquifer
    mes : str
        The measurement type to retrieve (e.g., electrical conductivity, piezometric level, temperature)
    save : str
        The user's choice to download the data ("YES" or "NO")
    """
    global OBS
    OBS = pd.DataFrame(columns=['Sensor', 'Latitude', 'Longitude', 'Datum', 'Value', 'Unit'])
    if json_data:
        providers = json_data["providers"]
        mes_exists = False  
        sensor_records = []    
        for provider in providers:
            sensors = provider["sensors"]
            for sensor in sensors:
                additional_info = sensor.get("componentAdditionalInfo", {})
                aquifer = additional_info.get("Aqüífer", "")
                mesura = sensor.get("description", "")
                if aquifer == aq:
                    if mesura == mes:
                        mes_exists = True  
                        component = sensor["component"]
                        component_desc = sensor["componentDesc"]
                        component_public_access = sensor["componentPublicAccess"]
                        sensor_name = sensor["sensor"]
                        sensor_location = sensor["location"]
                        lat, long = sensor_location.split()
                        sensor_lat = f"  Sensor Latitude: {lat}"
                        sensor_long = f"  Sensor Longitude: {long}"
                        sensor_description = sensor.get("description", "No description provided")
                        sensor_unit = sensor["unit"]
                        new_row = {
                            'Sensor': sensor_name,
                            'Latitude': float(lat),
                            'Longitude': float(long),
                            'Datum': "WGS84",
                            'Unit': sensor_unit
                        }
                        OBS = pd.concat([OBS, pd.DataFrame([new_row])], ignore_index=True)
                        point = Point(float(long), float(lat))
                        sensor_records.append({"Sensor": sensor_name, "geometry": point})   
                                           
                        # Parse response for value and update 'Head' column
                        url_sensor = f'https://aplicacions.aca.gencat.cat/sdim2/apirest/data/PIEZOMETRE-EST/{sensor_name}'
                        url_sensor = requests.get(url_sensor)
                        
                        json_ob = parse_response_ob(sensor_name, url_sensor)
                        value_ob(json_ob, sensor_name, url_sensor)     
        if not mes_exists:
            print('There are no existing measurements of "%s" for the selected aquifer' % (mes))
        else:
            display(OBS)
            if save == "SÍ":
                if mes == "Nivell piezomètric absolut":
                    folder_name = "observations_csv"
                    if not os.path.exists(folder_name):
                        os.makedirs(folder_name)
                    head_csv_name = os.path.join(folder_name, "obs_head.csv")
                    OBS.to_csv(head_csv_name, index=False)

                if mes == "Conductivitat":
                    folder_name = "observations_csv"
                    if not os.path.exists(folder_name):
                        os.makedirs(folder_name)
                    cond_csv_name = os.path.join(folder_name, "obs_cond.csv")
                    OBS.to_csv(cond_csv_name, index=False)

                if mes == "Temperatura de l'aigua":
                    folder_name = "observations_csv"
                    if not os.path.exists(folder_name):
                        os.makedirs(folder_name)
                    temp_csv_name = os.path.join(folder_name, "obs_temp.csv")
                    OBS.to_csv(temp_csv_name, index=False)
            else:
                pass    
            
            # Create GeoPandas DataFrame outside the loops
            gdf = gpd.GeoDataFrame(sensor_records, crs='EPSG:4326', geometry='geometry')

            # Display the map
            display(gdf.explore("Sensor", 
                                legend=False, 
                                marker_kwds={"fill":"False", "radius":7}, 
                                style_kwds={"stroke":False, "fillColor":"blue", "opacity":0.2}))
            

def reset_save_dropdown(change):
    """Resets the save option to "NO" when a new aquifer or measurement is selected.

    This function is triggered when the user selects a new aquifer or measurement 
    to ensure that the save option resets to the default value of "NO".

    Parameters
    ----------
    change : dict
        A dictionary containing the change information triggered by the user interaction.
    """
    sel_save.value = "NO"
    

#-------------------------------------------------------------------#
# Specification of the URL from which the data will be retrieved    #
#-------------------------------------------------------------------#
url = requests.get('https://aplicacions.aca.gencat.cat/sdim2/apirest/catalog?componentType=piezometre')
json_data = parse_response_catalog(url)  
    

#-------------------------------------------------------------------#
# Interactive Aquifer and Measurement Selection Tool                #
#-------------------------------------------------------------------#
aq_list = list_aquifers(json_data)
aq_list = list(aq_list)      
aq_list[0] = aq_list[0][1:]         
sel_aq = widgets.Dropdown(options=aq_list[0], value="Aqüífer profund del delta del Llobregat", 
                          description="Aquifer")
sel_mes = widgets.Dropdown(options=aq_list[1], value="Nivell piezomètric absolut", 
                           description="Measurement")
sel_save = widgets.Dropdown(options=['NO', 'YES'], value="NO", description="Download")
sel_aq.layout.width = '600px'
sel_mes.layout.width = '600px'
sel_save.layout.width = '600px'

sel_aq.observe(reset_save_dropdown, names='value')
sel_mes.observe(reset_save_dropdown, names='value')

out = widgets.interactive_output(to_print, {'aq': sel_aq, 'mes': sel_mes, 'save': sel_save, 
                                            'json_data': fixed(json_data)})
ui = widgets.VBox([sel_aq, sel_mes, sel_save])

display(ui, out)

Output()